Proposition 1 — Delayed Orders

Outer Query:

- Retrieves orders that were delivered after their required dates.
It compares ShipToDate and RequiredDate, calculates delay duration in days using DATEDIFF,
and lists orders ranked by the number of days delayed.

Overall Result:

- Displays which orders were delayed, how long they were late,
and highlights customers or logistics processes needing review.

In [ ]:
%%sql
SELECT o.OrderId,
       o.OrderDate,
       o.RequiredDate,
       o.ShipToDate,
       DATEDIFF(DAY, o.RequiredDate, o.ShipToDate) AS DelayDays
FROM Sales.[Order] AS o
WHERE o.ShipToDate > o.RequiredDate
ORDER BY DelayDays DESC;


UsageError: Cell magic `%%sql` not found.


Proposition 2 — Excessive Discounts

Outer Query:

- Finds employees granting the highest total discount values.
The discount value is computed as
DiscountPercentage × Quantity × UnitPrice.

Overall Result:

- Shows which employees offered the largest cumulative discounts,
helping management detect potential misuse or sales policy issues.


In [ ]:
%%sql
SELECT e.EmployeeId,
       e.EmployeeFirstName,
       e.EmployeeLastName,
       SUM(od.DiscountPercentage * od.Quantity * od.UnitPrice) AS TotalDiscountValue
FROM Sales.OrderDetail AS od
JOIN Sales.[Order] AS o ON od.OrderId = o.OrderId
JOIN HumanResources.Employee AS e ON o.EmployeeId = e.EmployeeId
GROUP BY e.EmployeeId, e.EmployeeFirstName, e.EmployeeLastName
ORDER BY TotalDiscountValue DESC;


Proposition 3 — Single-Order Customers

Outer Query:

- Aggregates all orders placed by each customer and filters those
who have made exactly one purchase.

Overall Result:

- Identifies one time buyers for potential follow up.

In [ ]:
%%sql
SELECT c.CustomerId,
       c.CustomerCompanyName,
       COUNT(o.OrderId) AS OrderCount
FROM Sales.Customer AS c
JOIN Sales.[Order] AS o ON c.CustomerId = o.CustomerId
GROUP BY c.CustomerId, c.CustomerCompanyName
HAVING COUNT(o.OrderId) = 1;


Proposition 4 — High-Price Suppliers

Outer Query:

- Calculates the average unit price of each supplier’s products.
Joins the Supplier and Product tables on SupplierId.

Overall Result:

- Ranks suppliers by their average product prices so the company can easily compare costs and see which ones charge more.


In [ ]:
%%sql
SELECT s.SupplierId,
       s.SupplierCompanyName,
       AVG(p.UnitPrice) AS AvgPrice
FROM Production.Supplier AS s
JOIN Production.Product AS p ON s.SupplierId = p.SupplierId
GROUP BY s.SupplierId, s.SupplierCompanyName
ORDER BY AvgPrice DESC;


Proposition 5 — Discontinued Products per Category

Outer Query:

- Counts discontinued products grouped by category.
Filters rows where Discontinued = 1.

Overall Result:

- Reveals which categories have the highest number of discontinued items
and may require catalog review or replacement.

In [ ]:
%%sql
SELECT c.CategoryName,
       COUNT(*) AS DiscontinuedCount
FROM Production.Product AS p
JOIN Production.Category AS c ON p.CategoryId = c.CategoryId
WHERE p.Discontinued = 1
GROUP BY c.CategoryName
ORDER BY DiscontinuedCount DESC;


Proposition 6 — Inactive Shippers

Outer Query:

- Lists all shippers and counts how many orders each handled.
Uses LEFT JOIN to include shippers with zero orders.

Overall Result:

- Helps logistics managers identify under utilized or inactive couriers.

In [ ]:
%%sql
SELECT s.ShipperCompanyName,
       COUNT(o.OrderId) AS OrdersHandled
FROM Sales.Shipper AS s
LEFT JOIN Sales.[Order] AS o ON s.ShipperId = o.ShipperId
GROUP BY s.ShipperCompanyName
ORDER BY OrdersHandled ASC;


Proposition 7 — Freight Cost Analysis

Outer Query:

- Analyzes freight performance by computing the number of orders,
average freight cost, and total freight handled per shipper.

Overall Result:

- Ranks shipping companies by average freight charge to locate
cost inefficiencies in distribution.

In [ ]:
%%sql
SELECT s.ShipperCompanyName,
       COUNT(o.OrderId) AS TotalOrders,
       AVG(o.Freight) AS AvgFreight,
       SUM(o.Freight) AS TotalFreight
FROM Sales.[Order] AS o
JOIN Sales.Shipper AS s ON o.ShipperId = s.ShipperId
WHERE o.Freight IS NOT NULL
GROUP BY s.ShipperCompanyName
ORDER BY AvgFreight DESC;
